In [51]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import data_gen as dg
import ising as isg

import torch
import geomloss

In [52]:
def gaussian_grad_potential(x, mu, L):
    diff = x - mu

    if diff.ndim == 1:
        diff = diff.unsqueeze(1)  # shape (d, 1)
        v1 = torch.linalg.solve_triangular(L, diff, upper=False)
        v = torch.linalg.solve_triangular(L.T, v1, upper=True)
        return v.squeeze(1)  # ritorna a shape (d,)
    else:
        # batch mode: x.shape = (n, d)
        v1 = torch.linalg.solve_triangular(L, diff.T, upper=False)
        v = torch.linalg.solve_triangular(L.T, v1, upper=True)
        return v.T


In [53]:
def langevin_step(x, mu, L, eps):

    noise = torch.randn_like(x)
    grad_V = gaussian_grad_potential(x, mu, L)
    
    return x - eps * grad_V + torch.sqrt(torch.tensor(2.0 * eps)) * noise

In [54]:
def evolve_one_sample(x, mu, L, eps, n_evolution):
    
    for _ in range(n_evolution):
        noise = torch.randn_like(x)
        grad_V = gaussian_grad_potential(x, mu, L)
        x = x - eps * grad_V + torch.sqrt(torch.tensor(2.0 * eps)) * noise

    return x

In [55]:
def evolve(samples, mu, L, eps=1e-2, n_evolution=10, seed=0):
    
    torch.manual_seed(seed)
    evolved_samples = []

    for i in range(samples.shape[0]):
        x = samples[i]
        evolved = evolve_one_sample(x, mu, L, eps, n_evolution)
        evolved_samples.append(evolved)

    return torch.stack(evolved_samples)

In [56]:
def lm_loss(samples, mu, L, eps=1e-2, n_evolution=10, seed=0):

    evolved_samples = evolve(samples, mu, L, eps=eps, n_evolution=n_evolution, seed=seed)
    loss_fn = geomloss.SamplesLoss("sinkhorn", p=2, blur=0.05)
    
    return loss_fn(samples, evolved_samples)

In [57]:
def optimize_lm(samples, n_steps=1000, lr=1e-2, eps=1e-2, n_evolution=10, seed=0, verbose=True):
    n_samples, d = samples.shape
    mu = torch.nn.Parameter(torch.zeros(d))
    L_raw = torch.nn.Parameter(torch.eye(d))

    optimizer = torch.optim.Adam([mu, L_raw], lr=lr)
    loss_history = []

    for step in range(n_steps):
        optimizer.zero_grad()
        L = torch.tril(L_raw) + torch.eye(d) * 1e-3
        loss = lm_loss(samples, mu, L, eps=eps, n_evolution=n_evolution, seed=seed)
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())
        if verbose and step % 1 == 0:
            print(f"Step {step} | Loss = {loss.item():.6f}")
    
    return {"mu": mu.detach(), "L": torch.tril(L_raw).detach()}, loss_history

In [58]:
def generate_gaussian_params(d, sigma_mu=0.1, sigma_cov=0.2, seed=0):
    torch.manual_seed(seed)
    mu = sigma_mu * torch.randn(d)
    A = sigma_cov * torch.randn(d, d)
    cov = A @ A.T + 1e-2 * torch.eye(d)
    return mu, cov

def generate_gaussian_data(mu, cov, n_samples, seed=0):
    torch.manual_seed(seed)
    L = torch.linalg.cholesky(cov)
    z = torch.randn(n_samples, mu.shape[0])
    return mu + z @ L.T

In [59]:
d = 5
n_samples = 500
seed = 0

mu_true, cov_true = generate_gaussian_params(d, seed=seed)
samples = generate_gaussian_data(mu_true, cov_true, n_samples, seed=seed+1)

print("Ottimizzazione in corso...")
result, history = optimize_lm(samples, n_steps=1000, lr=1e-2, eps=1e-2, n_evolution=10, seed=seed)

mu_hat = result["mu"]
L_hat = result["L"]
prec_hat = L_hat @ L_hat.T

cov_true_inv = torch.linalg.inv(cov_true)

print("\n---------- RISULTATI ----------")
print("mu_true:", mu_true)
print("mu_hat :", mu_hat)
print("\ncov^-1_true:\n", cov_true_inv)
print("prec_hat   :\n", prec_hat)

Ottimizzazione in corso...
Step 0 | Loss = 0.180140
Step 1 | Loss = 0.179333
Step 2 | Loss = 0.178496
Step 3 | Loss = 0.177623
Step 4 | Loss = 0.176709
Step 5 | Loss = 0.175753
Step 6 | Loss = 0.174753
Step 7 | Loss = 0.173706
Step 8 | Loss = 0.172611
Step 9 | Loss = 0.171465
Step 10 | Loss = 0.170268
Step 11 | Loss = 0.169016
Step 12 | Loss = 0.167710
Step 13 | Loss = 0.166348
Step 14 | Loss = 0.164931
Step 15 | Loss = 0.163457
Step 16 | Loss = 0.161927
Step 17 | Loss = 0.160342
Step 18 | Loss = 0.158706
Step 19 | Loss = 0.157020
Step 20 | Loss = 0.155286
Step 21 | Loss = 0.153505
Step 22 | Loss = 0.151688
Step 23 | Loss = 0.149841
Step 24 | Loss = 0.147965
Step 25 | Loss = 0.146055
Step 26 | Loss = 0.144100
Step 27 | Loss = 0.142099
Step 28 | Loss = 0.140009
Step 29 | Loss = 0.137808
Step 30 | Loss = 0.135484
Step 31 | Loss = 0.133015
Step 32 | Loss = 0.130418
Step 33 | Loss = 0.127744
Step 34 | Loss = 0.125054
Step 35 | Loss = 0.122419
Step 36 | Loss = 0.119928
Step 37 | Loss = 0.11

KeyboardInterrupt: 

In [ ]:
print("mu_true:", mu_true)
print("mu_hat :", mu_hat)
print(torch.linalg.norm(mu_true - mu_hat))
print("\ncov^-1_true:\n", cov_true_inv)
print("prec_hat   :\n", prec_hat)
print(torch.linalg.norm(cov_true_inv - prec_hat))

mu_true: tensor([ 0.1541, -0.0293, -0.2179,  0.0568, -0.1085])
mu_hat : tensor([ 0.0293,  0.2011, -0.2217, -0.0344, -0.3043])
tensor(0.3397)

cov^-1_true:
 tensor([[ 19.2456,  -9.9162,  16.2462,  -0.2003, -20.6090],
        [ -9.9162,  22.0264,  -7.1457,   0.1632,  25.2464],
        [ 16.2462,  -7.1457,  29.5353,  -0.4354, -20.0596],
        [ -0.2003,   0.1632,  -0.4354,   2.6278,   1.5867],
        [-20.6090,  25.2464, -20.0596,   1.5867,  43.5234]])
tensor(85.8763)
prec_hat   :
 tensor([[ 0.1948, -0.1337, -0.0559,  0.1384,  0.1696],
        [-0.1337,  0.2940, -0.0022, -0.0866, -0.2713],
        [-0.0559, -0.0022,  0.0567, -0.1305, -0.0087],
        [ 0.1384, -0.0866, -0.1305,  1.0431,  0.1293],
        [ 0.1696, -0.2713, -0.0087,  0.1293,  0.2988]])
